<h1>Análisis exploratorio a cuotas y estados </h1>
<p>Proyecto aplicado - Rescindidos Funeraria La Aurora</´p>
<p>Carlos Felipe Cortés Cataño - Carlos Luis Mora Cañas </p>

In [136]:
import pandas as pd
from datetime import datetime
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import re


# Estados

In [36]:
estados = pd.read_csv("../data/outputs/estados.csv")
estados = estados[["CodigoPrograma", "estado", "fecha"]]
estados.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CodigoPrograma,91397.0,NaN,NaN,NaN,85774.2069,56742.017271,52.0,31212.0,78460.0,139000.0,176178.0
estado,91397,58,Activo - Creacion de Solicitud,37100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fecha,91397,41758,2022-01-08 07:00:00.777,452,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
cuotas = pd.read_csv("../data/outputs/cuotas.csv")
cuotas.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CodigoPrograma,623745.0,NaN,NaN,NaN,50318.932735,44675.201487,56.0,15545.0,37335.0,60283.0,174604.0
factura,623745,401090,FEPE-3982,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
valorSinDescuento,623745.0,NaN,NaN,NaN,186910.137281,4179813.883796,1.0,17200.0,24200.0,72222.0,168829360.0
valorcondescuento,623745.0,NaN,NaN,NaN,176048.709146,4179053.939262,1.0,17200.0,24000.0,63720.0,168829360.0
cuotaSinDescuento,623745.0,NaN,NaN,NaN,19491.832578,22210.857617,-499454.1666,15000.0,17700.0,21700.0,4500000.0
cuotaConDescuento,623745.0,NaN,NaN,NaN,18367.845766,19584.782504,-499454.1666,13650.0,16700.0,20800.0,3825000.0
porcentajeDescuento,623745.0,NaN,NaN,NaN,5.068896,9.351622,0.0,0.0,0.0,10.0,2800.0
valorDescuento,623745.0,NaN,NaN,NaN,1124.37861,3761.511758,-35067.0,0.0,0.0,1420.0,675000.0
contrato,623745.0,NaN,NaN,NaN,1.748803,0.758765,1.0,1.0,2.0,2.0,6.0
contrato.1,623745.0,NaN,NaN,NaN,6.066312,3.591336,1.0,3.0,6.0,9.0,48.0


In [38]:
estados = pd.concat([estados.drop("estado",axis=1),\
    estados["estado"].str.split("-", expand=True, n=1)\
        .rename(columns={0:"estado", 1:"motivo"})], axis=1)

In [69]:
new_estados = estados.drop_duplicates("CodigoPrograma")["CodigoPrograma"]
new_estados.head()

0    55
1    56
2    58
3    72
4    73
Name: CodigoPrograma, dtype: int64

In [76]:
temp = estados.groupby(["CodigoPrograma", "estado"]
                       ).count().sort_values("fecha").reset_index()
temp["estado"] = temp["estado"].str.replace(" ", "")
new_estados = temp[temp["estado"] == "Activo"][["CodigoPrograma",
                                  "estado", "fecha"]]\
                                      .merge(new_estados, on="CodigoPrograma")\
                                          .rename(columns={"fecha": "qactivos"})


In [77]:
temp = estados.groupby(["CodigoPrograma", "estado"]
                       ).count().sort_values("fecha").reset_index()
temp["estado"] = temp["estado"].str.replace(" ", "")
new_estados = temp[temp["estado"] == "Inactivo"][["CodigoPrograma",
                                                "estado", "fecha"]]\
    .merge(new_estados, on="CodigoPrograma")\
    .rename(columns={"fecha": "qinactivos"})


In [81]:
new_estados =new_estados.drop(["estado_x", "estado_y"], axis=1)
new_estados.head()

,CodigoPrograma,qinactivos,qactivos
0,52755,1,1
1,52750,1,2
2,52768,1,1
3,52820,1,1
4,52808,1,2


Se ordenan para obtener la fecha mas reciente y la fecha final para activos e inactivos

In [93]:
new_estados = new_estados.merge(estados[estados["estado"] == "Activo "]
    .sort_values("fecha").drop_duplicates("CodigoPrograma", keep="first")\
        .drop("estado", axis=1).rename(columns={"fecha": "fechaactivoi",\
            "motivo":"motivoactivoi"}), on="CodigoPrograma")


In [94]:
new_estados = new_estados.merge(estados[estados["estado"] == "Inactivo "]
                                .sort_values("fecha").drop_duplicates("CodigoPrograma", keep="last")
                                .drop("estado", axis=1).rename(columns={"fecha": "fechainactivof",
                                                                        "motivo": "motivoinactivof"}), on="CodigoPrograma")


In [96]:
new_estados = new_estados.merge(estados[estados["estado"] == "Inactivo "]
                                .sort_values("fecha").drop_duplicates("CodigoPrograma", keep="first")
                                .drop("estado", axis=1).rename(columns={"fecha": "fechainactivoi",
                                                                        "motivo": "motivoinactivoi"}), on="CodigoPrograma")


In [ ]:
new_estados = new_estados.merge(estados[estados["estado"] == "Activo "]
                                .sort_values("fecha").drop_duplicates("CodigoPrograma", keep="last")
                                .drop("estado", axis=1).rename(columns={"fecha": "fechaactivof",
                                                                        "motivo": "motivoactivof"}), on="CodigoPrograma")


In [150]:
new_estados["fechaactivof"] = new_estados["fechaactivof"].apply(lambda x: datetime.strptime(
    x[:10], "%Y-%m-%d"))
new_estados["fechaactivoi"] = new_estados["fechaactivoi"].apply(lambda x: datetime.strptime(
    x[:10], "%Y-%m-%d"))
new_estados["fechainactivoi"] = new_estados["fechainactivoi"].apply(lambda x: datetime.strptime(
    x[:10], "%Y-%m-%d"))
new_estados["fechainactivof"] = new_estados["fechainactivof"].apply(lambda x: datetime.strptime(
    x[:10], "%Y-%m-%d"))
new_estados.head()


,CodigoPrograma,qinactivos,qactivos,fechaactivoi,motivoactivoi,fechainactivof,motivoinactivof,fechainactivoi,motivoinactivoi,fechaactivof,motivoactivof
0,52755,1,1,2015-10-07,Creacion de Solicitud,2021-12-07,Precio del plan,2021-12-07,Precio del plan,2015-10-07,Creacion de Solicitud
1,52750,1,2,2005-03-30,Creacion de Solicitud,2021-09-01,Inactivo para reactivación,2021-09-01,Inactivo para reactivación,2021-09-01,Activo
2,52768,1,1,2018-02-13,Creacion de Solicitud,2021-05-14,Dificultad para ubicarlo,2021-05-14,Dificultad para ubicarlo,2018-02-13,Creacion de Solicitud
3,52820,1,1,2017-04-26,Creacion de Solicitud,2022-09-17,Fallecimiento del titular,2022-09-17,Fallecimiento del titular,2017-04-26,Creacion de Solicitud
4,52808,1,2,2019-04-25,Creacion de Solicitud,2020-09-08,Incumplimiento en compromiso de pago,2020-09-08,Incumplimiento en compromiso de pago,2020-09-08,Activo


In [153]:
new_estados["inactivof_activoi"] = new_estados["fechainactivof"] - \
    new_estados["fechaactivoi"]
new_estados["inactivof_activof"] = new_estados["fechainactivof"] - \
    new_estados["fechaactivof"]
new_estados["inactivoi_activoi"] = new_estados["fechainactivoi"] - \
    new_estados["fechaactivoi"]
new_estados


,CodigoPrograma,qinactivos,qactivos,fechaactivoi,motivoactivoi,fechainactivof,motivoinactivof,fechainactivoi,motivoinactivoi,fechaactivof,motivoactivof,diassifai,diasifaf,diassiiai,inactivof_activoi,inactivof_activof,inactivoi_activoi
0,52755,1,1,2015-10-07,Creacion de Solicitud,2021-12-07,Precio del plan,2021-12-07,Precio del plan,2015-10-07,Creacion de Solicitud,2253 days,2253 days,2253 days,2253 days,2253 days,2253 days
1,52750,1,2,2005-03-30,Creacion de Solicitud,2021-09-01,Inactivo para reactivación,2021-09-01,Inactivo para reactivación,2021-09-01,Activo,5999 days,0 days,5999 days,5999 days,0 days,5999 days
2,52768,1,1,2018-02-13,Creacion de Solicitud,2021-05-14,Dificultad para ubicarlo,2021-05-14,Dificultad para ubicarlo,2018-02-13,Creacion de Solicitud,1186 days,1186 days,1186 days,1186 days,1186 days,1186 days
3,52820,1,1,2017-04-26,Creacion de Solicitud,2022-09-17,Fallecimiento del titular,2022-09-17,Fallecimiento del titular,2017-04-26,Creacion de Solicitud,1970 days,1970 days,1970 days,1970 days,1970 days,1970 days
4,52808,1,2,2019-04-25,Creacion de Solicitud,2020-09-08,Incumplimiento en compromiso de pago,2020-09-08,Incumplimiento en compromiso de pago,2020-09-08,Activo,502 days,0 days,502 days,502 days,0 days,502 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15934,124801,5,3,2021-08-31,Creacion de Solicitud,2022-07-25,Terminación y uso del contrato,2021-09-02,Pendiente primer pago,2022-03-02,Activo,328 days,145 days,2 days,328 days,145 days,2 days
15935,89101,5,3,2015-08-20,Creacion de Solicitud,2022-01-02,Inactivo rescindido,2021-05-10,Inactivo para reactivación,2022-01-02,Activo,2327 days,0 days,2090 days,2327 days,0 days,2090 days
15936,102639,6,3,2014-04-15,Creacion de Solicitud,2022-07-18,No Interesado,2021-06-07,Inactivo para reactivación,2022-04-23,Activo,3016 days,86 days,2610 days,3016 days,86 days,2610 days
15937,25473,6,4,2020-04-23,Creacion de Solicitud,2021-02-15,Programa con inconsistencia,2020-05-24,Pendiente primer pago,2020-10-17,Programa con inconsistencia,298 days,121 days,31 days,298 days,121 days,31 days


In [157]:
new_estados.to_csv("../data/outputs/estados_v2.csv", index=False)

# Recaudos

In [126]:
recaudos = pd.read_csv("../data/outputs/gestionrecaudo.csv")
recaudos = recaudos.dropna(subset=["fechaenvio", "fechaidealpago", "estado", "CodigoPrograma"])
recaudos.head(2)


,CodigoPrograma,fechaenvio,fechaidealpago,mensaje,estado,comentario
0,56,1900-01-01 00:00:00,2020-07-07 00:00:00,NaN,Exitosa,Gestión de recaudo exitosa
1,56,2020-08-15 05:00:00,2020-07-07 00:00:00,"linea personal no esta en uso, linea Alterna n...",Fallido,Gestión de recaudo no exitosa


In [127]:
recaudos = recaudos.drop(recaudos[recaudos["fechaidealpago"].str.isalpha()].index)
recaudos = recaudos.drop(
    recaudos[recaudos["fechaidealpago"].str.contains("a")].index)
recaudos = recaudos.drop(
    recaudos[recaudos["fechaidealpago"].apply(lambda x: len(x) < 10)].index)
recaudos = recaudos.drop(recaudos[recaudos["fechaenvio"].str.contains("a")].index)
recaudos = recaudos.drop(
    recaudos[recaudos["fechaenvio"].apply(lambda x: len(x) < 10)].index)
recaudos["fechaenvio"] = recaudos["fechaenvio"].apply(lambda x: datetime.strptime(
    x[:10], "%Y-%m-%d"))
recaudos["fechaidealpago"] = recaudos["fechaidealpago"].apply(lambda x: datetime.strptime(
    x[:10], "%Y-%m-%d"))
recaudos.head()

,CodigoPrograma,fechaenvio,fechaidealpago,mensaje,estado,comentario
0,56,1900-01-01,2020-07-07,NaN,Exitosa,Gestión de recaudo exitosa
1,56,2020-08-15,2020-07-07,"linea personal no esta en uso, linea Alterna n...",Fallido,Gestión de recaudo no exitosa
2,56,1900-01-01,2020-07-07,NaN,Inactivo,Gestión no ejecutada para el día planeado
3,56,1900-01-01,2020-07-07,se deja msj con Olga. tt linea personal fuer d...,Exitosa,Gestión de recaudo exitosa
4,58,1900-01-01,2020-08-10,NaN,Exitosa,Gestión de recaudo exitosa


In [130]:
recaudos["gestion"] = recaudos["fechaenvio"] - recaudos["fechaidealpago"]



In [143]:
recaudos

,CodigoPrograma,fechaenvio,fechaidealpago,mensaje,estado,comentario,gestion
0,56,1900-01-01,2020-07-07,NaN,Exitosa,Gestión de recaudo exitosa,-44017 days
1,56,2020-08-15,2020-07-07,"linea personal no esta en uso, linea Alterna n...",Fallido,Gestión de recaudo no exitosa,39 days
2,56,1900-01-01,2020-07-07,NaN,Inactivo,Gestión no ejecutada para el día planeado,-44017 days
3,56,1900-01-01,2020-07-07,se deja msj con Olga. tt linea personal fuer d...,Exitosa,Gestión de recaudo exitosa,-44017 days
4,58,1900-01-01,2020-08-10,NaN,Exitosa,Gestión de recaudo exitosa,-44051 days
...,...,...,...,...,...,...,...
886515,71368,2021-02-04,2021-01-26,"3102339468 tt o se contacta, pero recaudador e...",Fallido,Gestión de recaudo no exitosa,9 days
886516,71368,2020-08-03,2021-01-26,La Aurora te recuerda tener tu Previsión Exequ...,Exitosa,Gestión de recaudo exitosa,-176 days
886517,71368,1900-01-01,2021-01-26,},Fallido,Gestión de recaudo cambio en cuotas de carpet...,-44220 days
886518,120467,1900-01-01,2022-02-01,NaN,Exitosa,Gestión de recaudo exitosa,-44591 days


In [142]:
recaudos[recaudos["gestion"] < dt.timedelta(days=1)]


,CodigoPrograma,fechaenvio,fechaidealpago,mensaje,estado,comentario,gestion
0,56,1900-01-01,2020-07-07,NaN,Exitosa,Gestión de recaudo exitosa,-44017 days
2,56,1900-01-01,2020-07-07,NaN,Inactivo,Gestión no ejecutada para el día planeado,-44017 days
3,56,1900-01-01,2020-07-07,se deja msj con Olga. tt linea personal fuer d...,Exitosa,Gestión de recaudo exitosa,-44017 days
4,58,1900-01-01,2020-08-10,NaN,Exitosa,Gestión de recaudo exitosa,-44051 days
5,58,1900-01-01,2020-08-10,NaN,Inactivo,Gestión no ejecutada para el día planeado,-44051 days
...,...,...,...,...,...,...,...
886514,71368,1900-01-01,2021-01-26,NaN,Exitosa,Gestión de recaudo exitosa,-44220 days
886516,71368,2020-08-03,2021-01-26,La Aurora te recuerda tener tu Previsión Exequ...,Exitosa,Gestión de recaudo exitosa,-176 days
886517,71368,1900-01-01,2021-01-26,},Fallido,Gestión de recaudo cambio en cuotas de carpet...,-44220 days
886518,120467,1900-01-01,2022-02-01,NaN,Exitosa,Gestión de recaudo exitosa,-44591 days


In [149]:
recaudos["estado"].unique()

array(['Exitosa ', 'Fallido ', 'Inactivo ', 'Anulado '], dtype=object)

In [155]:
temp1 = recaudos[["CodigoPrograma", "estado"]]\
    [recaudos["estado"]== "Exitosa "]\
        .groupby(["CodigoPrograma"]).count().reset_index().rename(columns={"estado": "recauExitoso"})


In [156]:
temp2 = recaudos[["CodigoPrograma", "estado"]][recaudos["estado"] == "Fallido "]\
    .groupby(["CodigoPrograma"]).count().reset_index().rename(columns={"estado": "recauFallido"})


In [160]:
recaudoexp = temp1.merge(temp2, on="CodigoPrograma")
recaudoexp.head(2)

,CodigoPrograma,recauExitoso,recauFallido
0,56,2,1
1,59,6,31


In [161]:
recaudoexp.to_csv("../data/outputs/gestionrecaudo_v2.csv", index=False)

# Merge con base de datos principal

In [162]:
principal = pd.read_csv("../data/outputs/principal_v0.1.csv")
principal.head(2)

,CodigoPrograma,fechaSolicitud,tomador,edad,fechaNacimiento,#_inscritos_activos,#_mascotas_activas,Direccion,TipoPrograma,LocalidadVenta,EstadoActual,FechaRescindido,valorCuota_1,valorUltimaCuota,fechaIdealPago_CuotaCancelada,latitud,longitud
0,55,2018-09-07,Luz Dary Torres Cardona,53.0,1969-07-12,5.0,0.0,Calle 46 B - Carrera 9 # 99,Familiar,La Dorada,Inactivo - Inactivo rescindido,2019-06-14 07:00:02.857,13600.0,13600.0,2018-11-14,5.457571,-74.670340
1,56,2018-09-07,Leon Dario Cardona Velez,64.0,1958-09-28,0.0,0.0,Carrera 8 - Calle 17 # 28,Familiar,La Dorada,Inactivo - Influencia de seres cercanos,2020-07-23 05:00:00.000,7050.0,7050.0,2020-07-06,5.454073,-74.668154


In [164]:
pegarestados = pd.read_csv("../data/outputs/estados_v2.csv")
pegarestados.head(2)


,CodigoPrograma,qinactivos,qactivos,fechaactivoi,motivoactivoi,fechainactivof,motivoinactivof,fechainactivoi,motivoinactivoi,fechaactivof,motivoactivof,inactivof_activoi,inactivof_activof,inactivoi_activoi
0,52755,1,1,2015-10-07,Creacion de Solicitud,2021-12-07,Precio del plan,2021-12-07,Precio del plan,2015-10-07,Creacion de Solicitud,2253 days,2253 days,2253 days
1,52750,1,2,2005-03-30,Creacion de Solicitud,2021-09-01,Inactivo para reactivación,2021-09-01,Inactivo para reactivación,2021-09-01,Activo,5999 days,0 days,5999 days


In [167]:
pegarrecaudo = pd.read_csv("../data/outputs/gestionrecaudo_v2.csv")
pegarrecaudo.head(2)


,CodigoPrograma,recauExitoso,recauFallido
0,56,2,1
1,59,6,31


In [171]:
principal = principal.merge(pegarestados, on="CodigoPrograma").merge(pegarrecaudo, on="CodigoPrograma")

In [172]:
principal.to_csv("../data/outputs/principal_v0.2.csv")